In [1]:
%load_ext autoreload
%autoreload 2

import os; print(os.getcwd())
import socket; print(socket.gethostname())

from glob import glob
import Bio
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import numpy as np
import pandas as pd

import dnacauldron as dc
import seq_analysis
import re

/home/akonstantinova/prosculpt/codon_optimization
headnode


In [2]:
!/home/aljubetic/conda/envs/domesticator3/bin/python  /home/aljubetic/gits/domesticator3/domesticator3 --no_idt --help

usage: domesticator3 [-h] [--single_protein_fasta] [--nstruct NSTRUCT]
                     [--max_tries MAX_TRIES] [--no_idt]
                     [--idt_credentials_dir IDT_CREDENTIALS_DIR]
                     [--idt_threshold IDT_THRESHOLD] [--no_opt]
                     [--ramp_kmers_boost RAMP_KMERS_BOOST] [--version]
                     proteins [proteins ...] vector

A sophisticated codon optimizer for the discerning protein designer

positional arguments:
  proteins              Either one or more fasta files containing one or more
                        protein sequences or one or more pdb files
  vector                A genbank (.gb) file containing annotations in the
                        domesticator format to control domesticator function

options:
  -h, --help            show this help message and exit
  --single_protein_fasta
                        Assign increasing chain letters in the order that they
                        appear in the file. Lettering resets b

In [ ]:
prosculpt_dir = os.getcwd()

#working directory for codon optimization
base_dir = '/home/akonstantinova/projects/2025/2025_06_30_round3_domesticator' 
out_dir = base_dir+'/out/'
fasta_file = base_dir + '/proteins.fasta'

#vector file used for codon optimization
vector_file= prosculpt_dir + '/ggv2.gb'

#vectors for running golden gate
vectors_path = prosculpt_dir + '/golden_gate_vectors'
insert_path = out_dir + 'order.dna.fasta'

In [ ]:
if not os.path.exists(out_dir):
    os.system(f'mkdir -v {out_dir}')

os.chdir(out_dir)
cmd = f'/home/aljubetic/conda/envs/domesticator3/bin/python  /home/aljubetic/gits/domesticator3/domesticator3  {fasta_file} {vector_file} --nstruct 10 --no_idt'
print(cmd)
os.system(cmd)


/home/akonstantinova/projects/2025/2025_06_30_round3_domesticator
/home/akonstantinova/projects/2025/2025_06_30_round3_domesticator/out/
mkdir: created directory '/home/akonstantinova/projects/2025/2025_06_30_round3_domesticator/out/'


In [4]:
def create_assembly(vector, insert):
    repository = dc.SequenceRepository(collections={"parts": {vector.id: vector, insert.id: insert}})
    assembly = dc.Type2sRestrictionAssembly(parts=[vector.id, insert.id], enzyme='BsaI')
    simulation = assembly.simulate(sequence_repository=repository)
    assembly_record = simulation.construct_records[0]
    assembly_record.name = insert.id+'_'+vector.id
    return assembly_record

def create_translation(assembly_record):
    #translation
    fullseq  = assembly_record.seq

    #search for T7 in the full sequence and rearrange the sequence to avoid cutting the transcript
    t7 = 'taatacgactcactataggggaattgtgagcggataacaattcccctctagaaataattttgtttaactttaagaaggagatatacat'.upper()

    #check that t7 is only in the sequence once
    instances = [m.end() for m in re.finditer(t7, str(fullseq))]
    if len(instances)>1:
        raise Exception('t7 in the sequence more than once, might cause ambiguity in translation')

    t7_end = instances[0]

    fullseq = fullseq[t7_end:]+fullseq[:t7_end]
    fullseq_aa = fullseq.translate()
    stop_codon = fullseq_aa.find('*')
    good_aa = fullseq_aa[:stop_codon]
    good_dna = fullseq[:stop_codon*3]
    return good_aa, good_dna

def get_properties(good_aa, assembly_record):
    protein_properties = seq_analysis.analyse_sequence(good_aa, name = assembly_record.name)
    return protein_properties

In [10]:
insert_list = []
for  record in SeqIO.parse(insert_path, "fasta"):
    record.annotations['topology'] = 'circular'
    insert_list.append(record)

insert_list

[SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='6B6FN', name='6B6FN', description='6B6FN', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='7HN5G', name='7HN5G', description='7HN5G', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='7ODDK', name='7ODDK', description='7ODDK', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='A4Z57', name='A4Z57', description='A4Z57', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='AQQG3', name='AQQG3', description='AQQG3', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='B44J3', name='B44J3', description='B44J3', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGGCGCGCTAGCGCGCGG...GAG'), id='BT4PQ', name='BT4PQ', description='BT4PQ', dbxre

In [ ]:
properties_list = []

#either can specify a vector to clone into, or go through all vectors in the folder?
# vectors_to_use = ['er12_pet-41a_mskek_spydersilk_16gs_agga_ccdb_ctcg_his_trp.gb']
vectors_to_use = os.listdir(vectors_path)

for vec in os.listdir(vectors_path):
    if vec in vectors_to_use:
        print(vec)
        for  record in SeqIO.parse(vectors_path+'/'+vec, "genbank"):
            vector = record
            for insert in insert_list:
                try:
                    assembly_record = create_assembly(vector, insert)
                    good_aa, good_dna = create_translation(assembly_record)
                    print(good_aa)
                    protein_properties= get_properties(good_aa, assembly_record)
                    protein_properties['dna_sequence']=good_dna
                    properties_list.append(protein_properties)

                except IndexError:
                    print('there has been an error with the assembly ', insert.id)
                    continue


er12_pet-41a_mskek_spydersilk_16gs_agga_ccdb_ctcg_his_trp.gb
MSKEKIANSPFSNPNTAEAFARSFVSNIVSSGEFGAQGAEKFDDIIQSLIQAQSMGKGRHDTKADAKAMQVALASSIAELVIAESSGGDVQRKTNVISNALRNALMSTTGSPNEEFVHEVQDLIQMLSQEQINEVGSGGGSGGGGSGGSSGKEMTLNITVDKIEQMPKVKELIDEWIKYAEEKGWNVTVNVTINLGGSGSHHHHHHHHW
MSKEKIANSPFSNPNTAEAFARSFVSNIVSSGEFGAQGAEKFDDIIQSLIQAQSMGKGRHDTKADAKAMQVALASSIAELVIAESSGGDVQRKTNVISNALRNALMSTTGSPNEEFVHEVQDLIQMLSQEQINEVGSGGGSGGGGSGGSSGERLEVSFSATITDREDLEKLVETIKELTAAGASVSVSLTMSLELAIEFMKAVKDLKGLSLSLTVGGSGSHHHHHHHHW
MSKEKIANSPFSNPNTAEAFARSFVSNIVSSGEFGAQGAEKFDDIIQSLIQAQSMGKGRHDTKADAKAMQVALASSIAELVIAESSGGDVQRKTNVISNALRNALMSTTGSPNEEFVHEVQDLIQMLSQEQINEVGSGGGSGGGGSGGSSGEKEEARLELIRELLELAKKSNLEVAKEIMKLANWLMEEVVKEGGVEEVKRLNEILREIIGSGSHHHHHHHHW
MSKEKIANSPFSNPNTAEAFARSFVSNIVSSGEFGAQGAEKFDDIIQSLIQAQSMGKGRHDTKADAKAMQVALASSIAELVIAESSGGDVQRKTNVISNALRNALMSTTGSPNEEFVHEVQDLIQMLSQEQINEVGSGGGSGGGGSGGSSGKVKVTVTINVEAEPEKLKEALEIIKELTEEFLEYYKDYELTININLYLRWGSGSHHHHHHHHW
MSKEKIANSPFSNPNTAEAFARSFVSNIVSSGEFGAQGAEKFDDIIQSLIQAQSMGKGRH

/home/akonstantinova/anaconda3/lib/python3.11/site-packages/Bio/SeqFeature.py:1040: BiopythonParserWarning: Attempting to fix invalid location '5454..139' as it looks like incorrect origin wrapping. Please fix input file, this could have unintended behavior.
  warnings.warn(
/home/akonstantinova/anaconda3/lib/python3.11/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [10]:
with open('assemblies_aa.fasta', 'w') as f:
    for i in properties_list:
        f.write(f">{i['name']}\n")
        f.write(f"{i['sequence']}\n")

with open('assemblies_dna.fasta', 'w') as f:
    for i in properties_list:
        f.write(f">{i['name']}\n")
        f.write(f"{i['dna_sequence']}\n")

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(properties_list)
df.to_csv('protein_properties.csv', index=False)

In [ ]:
#get back to the base directory 
os.chdir(base_dir)
os.getcwd()

'/home/akonstantinova/projects/2025/2025_06_30_round3_domesticator/out'